In [7]:
import cv2
import logging
import numpy
import typing
from utils import create_folder, get_logger, debounce, throttle
import time



下面是主程序


In [8]:

logger: logging.Logger = get_logger(__name__)

def get_homography_martix(points_1, point2_2):
    A: numpy.ndarray[int] = numpy.zeros(shape=( len(points_1) * 2 ,  9), dtype=int)
    for i in range(len(points_1)):
        x1, y1 = points_1[i]
        x2, y2 = point2_2[i]
        # -x1 * h11 - y1 * h12 - h13 + x2 = 0
        # -x1 * h21 - y1 * h22 - h23 + y2 = 0
        idx = i + i
        A[idx, 0] = -x1
        A[idx, 1] = -y1
        A[idx, 2] = -1
        A[idx, 3:6] = 0
        A[idx, 6] = x1 * x2
        A[idx, 7] = y1 * x2
        A[idx, 8] = x2
        idx += 1
        A[idx, 0:3] = 0
        A[idx, 3] = -x1
        A[idx, 4] = -y1
        A[idx, 5] = -1
        A[idx, 6] = x1 * y2
        A[idx, 7] = y1 * y2
        A[idx, 8] = y2

    _, _, Vt = numpy.linalg.svd(A)

    H = Vt[-1].reshape(3, 3)
    return H

# Load the images h1.jpg and h2.jpg
image1 = cv2.imread('h1.jpeg')
image2 = cv2.imread('h2.jpeg')
image2 = cv2.resize(src=image2, dsize=image1.shape[:2])

# Create lists to store selected points for h1.jpg and h2.jpg
points_h1 = []
points_h2 = []

# Function to handle mouse events
@throttle(0.5)
def select_points(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN or event == cv2.EVENT_RBUTTONDOWN:
        point_list = param.get("point_list")
        if len(point_list) < 4:
            cv2.circle(param.get("image"), (x, y), 3, (255, 0, 0), 2)
            point_list.append((x,y))

# Create windows to display images and set mouse event handlers
cv2.namedWindow('h1.jpeg')
cv2.setMouseCallback('h1.jpeg', 
                     select_points, 
                     {"image": image1,
                      "point_list": points_h1})
cv2.namedWindow('h2.jpeg')
cv2.setMouseCallback('h2.jpeg', 
                     select_points, 
                     {"image": image2, 
                      "point_list": points_h2})

while True:
    # time.sleep(200)
    cv2.imshow('h1.jpeg', image1)
    cv2.imshow('h2.jpeg', image2)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


# Close OpenCV windows
# cv2.destroyAllWindows()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [9]:


# Display the selected points
logger.info(f"points_h1:{points_h1}")
logger.info(f"points_h2:{points_h2}")

image1 = cv2.imread('h1.jpeg')
image2 = cv2.imread('h2.jpeg')
image2 = cv2.resize(src=image2, dsize=image1.shape[:2])

H_1_to_2 = get_homography_martix(points_1=points_h1, point2_2=points_h2)
H_2_to_1 = get_homography_martix(points_1=points_h2, point2_2=points_h1)
# H_1_to_2_estimate = cv2.findHomography(random_points_image1, random_points_image2, cv2.RANSAC, ransac_threshold)[0]
# image2_height, image2_width = image2.shape
# result = cv2.warpPerspective(src=image1, M=H_1_to_2, dsize=(image2.shape[:2]), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=[0,0,0])
result = cv2.warpPerspective(src=image1, M=H_1_to_2, dsize=(image2.shape[:2]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=[0,0,0])

cv2.imwrite('warpPerspective_homography_image1.png', result)
result = cv2.warpPerspective(src=image2, M=H_2_to_1, dsize=(image1.shape[:2]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=[0,0,0])
cv2.imwrite('warpPerspective_homography_image2.png', result)


custom_result = numpy.zeros(shape=image2.shape, dtype=image2.dtype)
image2_height, image2_width, _ = image2.shape
image1_height, image1_width, _ = image1.shape
for y in range(image2_height):
    for x in range(image2_width):
        p = numpy.dot(H_1_to_2, numpy.array([y, x, 1]))
        p = p / p[2]
        target_y, target_x = int(p[0]), int(p[1])
        if 0 <= target_x < image1_width and 0 <= target_y < image1_height:
            custom_result[y, x] = image1[target_y, target_x]
        else:
            custom_result[y, x] = [0, 0 ,0]
cv2.imwrite('custom_homography_image1.png', custom_result)

custom_result = numpy.zeros(shape=image1.shape, dtype=image1.dtype)

for y in range(image1_height):
    for x in range(image1_width):
        p = numpy.dot(H_2_to_1, numpy.array([y, x, 1]))
        p = p / p[2]
        target_y, target_x = int(p[0]), int(p[1])
        if 0 <= target_x < image2_width and 0 <= target_y < image2_height:
            custom_result[y, x] = image2[target_y, target_x]
        else:
            custom_result[y, x] = [0,0,0]
cv2.imwrite('custom_homography_image2.png', custom_result)










2023-10-01 03:16:12,681 - __main__ - INFO - points_h1:[(26, 23), (1149, 30), (1138, 1149), (43, 1151)]
2023-10-01 03:16:12,681 - __main__ - INFO - points_h1:[(26, 23), (1149, 30), (1138, 1149), (43, 1151)]
2023-10-01 03:16:12,681 - __main__ - INFO - points_h1:[(26, 23), (1149, 30), (1138, 1149), (43, 1151)]
INFO:__main__:points_h1:[(26, 23), (1149, 30), (1138, 1149), (43, 1151)]
2023-10-01 03:16:12,687 - __main__ - INFO - points_h2:[(22, 429), (661, 20), (1175, 593), (521, 1100)]
2023-10-01 03:16:12,687 - __main__ - INFO - points_h2:[(22, 429), (661, 20), (1175, 593), (521, 1100)]
2023-10-01 03:16:12,687 - __main__ - INFO - points_h2:[(22, 429), (661, 20), (1175, 593), (521, 1100)]
INFO:__main__:points_h2:[(22, 429), (661, 20), (1175, 593), (521, 1100)]


True